![](https://www.math.unipd.it/~marcuzzi/BannerStrumentifondamentali.png)

# cuBLAS

La libreria cuBLAS è un'implementazione della libreria BLAS (Basic Linear Algebra Subprograms) su scheda grafica fornita da NVIDIA. Il binding a cuBLAS di Pyculib fornisce un'interfaccia che accetta array NumPy trasferendoli automaticamente sulla memoria device, oppure array Numba precedentemente allocati sulla GPU.

Tutte le funzioni sono accessibili tramite gli oggetti della classe `pyculib.blas.Blas`.
     
     
# Attenzione
La libreria cuBlas assume sempre che gli array siano salvati **per colonne** (come in fortran), e non per righe (come in C e Python). 
![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Row_and_column_major_order.svg/170px-Row_and_column_major_order.svg.png)
Per ovviare a questo problema, è sufficiente dichiarare gli array numpy specificando `order = 'F'` (cioè che siano salvati in memoria nell'ordine di Fortran, per righe), oppure passando alle funzioni gli array modificati con `numpy.asfortranarray()`.

In [1]:
import numpy as np
import numba
from numba import cuda
import math

In [2]:
import pyculib
import pyculib.blas as blas

In [3]:
# creo oggetto BLAS
cublasH = blas.Blas()

## BLAS level 1: operazioni vettore-vettore

In [4]:
# generazione dei dati
n = 256
x = np.random.random((n,))
y = np.random.random((n,))

In [5]:
# norma 2
print('cublas: ',cublasH.nrm2(x), 'numpy: ', np.linalg.norm(x))

cublas:  8.986256263249352 numpy:  8.986256263249352


In [6]:
# prodotto interno
print('cublas: ', cublasH.dot(x,y), 'numpy: ', np.dot(x,y))

cublas:  57.7902185340501 numpy:  57.7902185340501


In [8]:
# scal(alpha, x)
# moltiplicazione per scalare sul posto x = alpha*x
alpha = 4.0
xold = x.copy()
cublasH.scal(alpha, x)
print('||x-a*xold|| = ',cublasH.nrm2(x-alpha*xold))

||x-a*xold|| =  0.0


In [9]:
# axpy(alpha, x, y)
# calcola sul posto  y = alpha * x + y 
yold = y.copy()
cublasH.axpy(alpha, x, y)
print('||a*x + yold - y|| = ',cublasH.nrm2(alpha*x + yold - y))

||a*x + yold - y|| =  0.0


In [10]:
# amax(x)
# calcola l'indice dell'elemento piu' grande di un array
print('cublas: ', cublasH.amax(x), 'numpy: ',np.argmax(x))

cublas:  174 numpy:  174


In [11]:
# amin(x)
# calcola l'indice dell'elemento piu' piccolo di un array
print('cublas: ', cublasH.amin(x), 'numpy: ',np.argmin(x))

cublas:  221 numpy:  221


In [12]:
# asum(x)
# calcola la somma di tutti gli elementi in un array
print('cublas: ', cublasH.asum(x) , 'numpy: ',np.sum(x))

cublas:  492.9727590689481 numpy:  492.9727590689482


In [13]:
# rot(x, y, c, s)
# applica sul posto la matrice di rotazione di Givens
# |  c s |
# | -s c |
# agli array x e y, i.e. la rotazione nel piano x,y in senso orario di angolo alpha dato da 
# cos(alpha) = c, sin(alpha) = s.
# equivalente a x, y = c * x + s * y, -s * x + c * y

a = math.pi/4
c = math.cos(a)
s = math.sin(a)
xold = x.copy()
yold = y.copy()
cublasH.rot(x, y, c, s)

print('||c * xold + s * yold - x||  = ', cublasH.nrm2(c * xold + s * yold - x))
print('||-s * xold + c * yold - y|| = ', cublasH.nrm2(-s * xold + c * yold - y))

||c * xold + s * yold - x||  =  6.973225884473334e-15
||-s * xold + c * yold - y|| =  7.513973108660854e-15


In [14]:
# rotg(a, b)
# costruisce la matrice di rotazione di Givens che annulla la seconda entrata di un vettore colonna (a,b)^T, 
# cioe' risolve
# |  c s | | a | = | r |
# | -s c | | b |   | 0 |
# dove c**2+s**2 = 1 e a**2+b**2 = r.

# restuisce una tupla (r, z, c, s) dove
# r = a**2 + b**2
# z puo' essere usato per ricostruire c ed s (vedi documentazione cublas)
# c coseno
# s seno

a = 3.7
b = 5.2
r, z, c, s = cublasH.rotg(a,b)

print('c * a + s * b   = ', c * a + s * b, ', r = ', r)
print('- s * a + c * b = ', - s * a + c * b)

c * a + s * b   =  6.3820059542435414 , r =  6.38200595424354
- s * a + c * b =  4.440892098500626e-16


# Blas level 2: operazioni matrice-vettore

Nelle routine di livello 2 e 3 è necessario specificare alcune caratteristiche della matrice in input attraverso i seguenti argomenti 

- trans specifica l'operazione da effettuare su una matrice $X$
    - `trans = 'N'` corrisponde a op($X$) = $X$
    - `trans = 'T'` corrisponde a op($X$) = $X^T$ (trasposta)
    - `trans = 'H'` corrisponde a op($X$) = $X^H$ (trasposta coniugata)
    
- uplo
    - `uplo = 'L'` indica che la matrice è tringolare inferiore
    - `uplo = 'U'` indica che la matrice è tringolare superiore

In [15]:
# generazione dei dati singola precisione
m = n - 128
x = np.random.random((n,)).astype(np.float32)
y = np.zeros((m,)).astype(np.float32)
A = np.random.random((m,n)).astype(np.float32) 

In [16]:
# gemv(trans, m, n, alpha, A, x, beta, y)
# moltiplicazione matrice-vettore y = alpha * op(A) * x + beta * y
# trans determina op(A) e dim(A) = (m,n)
# Attenzione, A deve essere passata come array Fortran
alpha = 1.0
beta  = 0.0

cublasH.gemv('N', m, n, alpha, np.asfortranarray(A), x, beta, y)

In [17]:
# l'errore e' pari circa alla precisione di macchina con i float 32
print('||Ax - y||  = ', np.linalg.norm( A@x - y))

||Ax - y||  =  7.3178584e-05


In [18]:
# generazione dei dati doppia precisione
x = np.random.random((n,)).astype(np.float64)
A = np.random.random((n,n)).astype(np.float64) 

In [19]:
# trmv(uplo, trans, diag, n, A, x)
# moltiplicazione matrice triagolare-vettore sul posto x = op(A) * x
# uplo indica se la matrice e' triangolare inferiore o superiore, trans determina op(A), e dim(A) = (n,n)
xold = x.copy()
cublasH.trmv('U', 'N', False, n, np.asfortranarray(A), x)

In [20]:
# l'errore e' pari circa alla precisione di macchina con i float 64
print('||Ax - y||  = ', np.linalg.norm( np.triu(A)@xold - x))

||Ax - y||  =  2.4731274494190633e-13


# Blas level 3: operazioni matrice-matrice

In [21]:
# generazione dei dati
m = n - 128
k = m
A = np.random.random((m,k)).astype(np.float64) 
B = np.random.random((k,n)).astype(np.float64) 
C = np.zeros((m,n), dtype = np.float64)
C = np.asfortranarray(C)

In [22]:
# gemm(transa, transb, m, n, k, alpha, A, B, beta, C)
# moltiplicazione matrice-matrice C = alpha * op(A) * op(B) + beta * C
# transa determina op(A), transb determina op(B), dim(A) = (m,k), dim(A) = (k,n) e dim(C) = (m,n)
cublasH.gemm('N', 'N', m, n, k, alpha, np.asfortranarray(A), np.asfortranarray(B), beta, C)

In [23]:
print('||AB - C||  = ', np.linalg.norm(A@B - C))

||AB - C||  =  5.742003074354213e-13


In [24]:
A = np.random.random((m,n)).astype(np.float64) 
B = np.random.random((m,n)).astype(np.float64) 
C = np.zeros((m,n), dtype = np.float64)
C = np.asfortranarray(C)

In [25]:
# geam(transa, transb, m, n, alpha, A, beta, B, C)
# addizione/trasposizione di matrici C = alpha * op(A) + beta * op(B)
# transa determina op(A), transb determina op(B), dim(A) = dim(B) = dim(C) = (m,n)
cublasH.geam('N', 'N', m, n, alpha, np.asfortranarray(A), alpha, np.asfortranarray(B), C)

In [26]:
print('||A+B - C||  = ', np.linalg.norm(A+B - C))

||A+B - C||  =  0.0
